# Классификаторы на основе деревьев принятия решений

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

from sklearn.tree import export_graphviz
import subprocess

from tqdm import tqdm
from collections import Counter

import pymorphy2
import re

### Пример дерева принятия решения

<img src="IkBzK.png">

## Как определяются лучшие разбиения (best splits)?
### Меры неопределенности (impurity measures)

Пусть $p_k$ - это доля класса $C_k$ в узле дерева $S$.

1. Missclassification error  
$$I(S) = 1 - \max\limits_k p_k $$
2. Gini index 
$$I(S) = 1 - \sum\limits_k (p_k)^2 = \sum\limits_{k'\neq k} p_{k'} p_k$$
3. Entropy 
$$I(S) = -\sum\limits_k p_k \log(p_k)$$


### Алгоритмы построения деревьев
 
** ID 3 **
* Только категориальные признаки
* Количество потомков = количеству значений признака
* Строится до максимальной глубины

** С 4.5 **
* Поддержка вещественных признаков
* Категриальные как в ID3
* При пропуске значения переход по всем потомкам
* Удаляет избыточные ветвления

** СART **
* Специальная процедура усещения дерева после построения (post prunning)

### Преимущества и недостатки

** Преимущества **
* Простота построения
* Интерпретируемость (при небольшой глубине)
* Требуются минимальная предобработка признаков
* Встроенный отбор признаков

** Недостатки **
* Границы строяется только параллельно или перпендикулярно осям
* При изменении набора данных надо полностью перестраивать и результат может получится совершенно иным
* Жадность построения


In [2]:
# Спасибо Андрею Шестакову за пример и демонстрашку.

def demo_dec_tree(depth=1):
    fig, ax = plt.subplots(1,2)
    fig.set_figheight(5)

    np.random.seed(0)

    C = np.array([[0., -0.7], [1.5, 0.7]])
    gauss1 = np.dot(np.random.randn(200, 2) + np.array([4, 2]), C)
    gauss2 = np.dot(np.random.randn(300, 2), C)

    X = np.vstack([gauss1, gauss2])
    y = np.r_[np.ones(200), np.zeros(300)]

    ax[1].scatter(X[:,0], X[:, 1], c=y)
    ax[1].set_xlabel('$x_1$')
    ax[1].set_ylabel('$x_2$')

    # Dec Tree Stuff
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=123)
    tree.fit(X,y)

    x_range = np.linspace(X.min(), X.max(), 100)
    xx1, xx2 = np.meshgrid(x_range, x_range)

    Y = tree.predict(np.c_[xx1.ravel(), xx2.ravel()])
    Y = Y.reshape(xx1.shape)

    ax[1].contourf(xx1, xx2, Y, alpha=0.3)
    ax[1].scatter(X[:,0], X[:,1],c=y)
    
#     dot_data = StringIO()  
#     tree.export_graphviz(tree, out_file=dot_data,  
#                      filled=True, rounded=True,  
#                      special_characters=True)  
#     graph = pydot.graph_from_dot_data(dot_data.getvalue())  
#     ax[0].imshow(graph[0].create_png())


    with open('tree.dot', 'w') as fout:
        export_graphviz(tree, out_file=fout, feature_names=['x1', 'x2'], class_names=['0', '1'])
    command = ["dot", "-Tpng", "tree.dot", "-o", "tree.png"]
    subprocess.check_call(command)
    ax[0].imshow(plt.imread('tree.png'))
    ax[0].axis("off")
    
    plt.show()

In [3]:
try:
    fig = interact(demo_dec_tree, depth=IntSlider(min=1, max=5, value=1))
except:
    print('Что-то не так. Посмотрите на доску')

Возьмем какое-то количество научных статей разной тематики и сформируем из них набор данных. Нашей задачей будет классифицировать статьи по разделам науки.

** Внимание!!! **
Исходные файлы со статьями, из которых проводилась выборка не предоставляются! В связи с этим код ниже работать не будет.

In [4]:
path="/home/edward/projects/TEST/getRusCorpora/Python/"
files=["all_cyberleninka_archi2.txt", "all_cyberleninka_arts2.txt", "all_cyberleninka_auto2.txt", "all_cyberleninka_bio2.txt", "all_cyberleninka_chemystry2.txt", "all_cyberleninka_cyber2.txt"]

classes=[]
texts=[]

for c, name in enumerate(files):
    print(name)
    with open(path+name) as fil:
        txt=fil.read()
        arts=txt.split("\n=====\n")
        texts.extend(arts[1:51])
        classes.extend([c for i in range(50)])
        
txt=""
arts=[]

with open("cyber_dataset.txt", "wt") as fil:
    fil.write(" ".join([str(c) for c in classes])+"\n=====\n")
    for text in texts:
        fil.write(text)
        fil.write("\n=====\n")
    

all_cyberleninka_archi2.txt
all_cyberleninka_arts2.txt
all_cyberleninka_auto2.txt
all_cyberleninka_bio2.txt
all_cyberleninka_chemystry2.txt
all_cyberleninka_cyber2.txt


Возьмем файл со сформированным набором данных и прочитаем статьи в память. После этого сформируем частотные словари для статей.

In [5]:
def getArticleDictionary(morph, text):
    words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    reswords=[]

    for w in words:
        wordform=morph.parse(w)[0]
        try:
            if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'INFN']:
                reswords.append(wordform.normal_form)
        except:
            pass

    stat=Counter(reswords)
    # Берем только слова с частотой больше 1.
    stat={a: stat[a] for a in stat.keys() if stat[a]>1}
    return stat



In [6]:
morph=pymorphy2.MorphAnalyzer()

In [7]:
with open("cyber_dataset.txt", "rt") as fil:
    text=fil.read()
articles=text.split("\n=====\n")
classes=articles[0].split(' ')
dicts=[]
for art in tqdm(articles[1:-1]):
    dicts.append(getArticleDictionary(morph, art))


100%|██████████| 300/300 [00:20<00:00, 14.96it/s]


In [8]:
tree = DecisionTreeClassifier(criterion='entropy', random_state=333)

In [9]:
all_words={}
for d in dicts:
    for word in d.keys():
        if word not in all_words.keys():
            all_words[word]=len(all_words.keys())

In [10]:
#words=list([list(d.keys()) for d in dicts])
words=[np.array([1 if w in d.keys() else 0 for w in all_words.keys()]) for d in dicts]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(words, classes, test_size=0.2)


In [12]:
tree.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=333, splitter='best')

In [13]:
y_hat=tree.predict(X_test)

In [14]:
list(zip(y_hat, y_test))

[('3', '3'),
 ('3', '3'),
 ('5', '5'),
 ('5', '5'),
 ('0', '2'),
 ('4', '4'),
 ('3', '1'),
 ('1', '1'),
 ('1', '1'),
 ('4', '4'),
 ('3', '3'),
 ('4', '4'),
 ('1', '1'),
 ('2', '2'),
 ('1', '0'),
 ('5', '2'),
 ('5', '5'),
 ('1', '0'),
 ('1', '1'),
 ('3', '3'),
 ('4', '2'),
 ('1', '1'),
 ('3', '4'),
 ('5', '2'),
 ('3', '3'),
 ('1', '1'),
 ('3', '3'),
 ('5', '5'),
 ('0', '0'),
 ('4', '3'),
 ('1', '0'),
 ('3', '3'),
 ('0', '5'),
 ('5', '2'),
 ('1', '1'),
 ('0', '0'),
 ('5', '5'),
 ('2', '2'),
 ('4', '4'),
 ('3', '3'),
 ('0', '0'),
 ('4', '4'),
 ('0', '2'),
 ('5', '5'),
 ('1', '1'),
 ('0', '0'),
 ('3', '4'),
 ('3', '1'),
 ('1', '1'),
 ('5', '5'),
 ('0', '3'),
 ('1', '0'),
 ('5', '5'),
 ('5', '5'),
 ('0', '0'),
 ('4', '5'),
 ('3', '3'),
 ('3', '3'),
 ('5', '5'),
 ('2', '2')]

In [15]:
len([1 for i, j in zip(y_hat, y_test) if i==j])/len(y_test)

0.7

In [16]:
confusion_matrix(y_test, y_hat)

#max_depth
#criterion='entropy',

array([[ 5,  4,  0,  0,  0,  0],
       [ 0,  9,  0,  2,  0,  0],
       [ 2,  0,  3,  0,  1,  3],
       [ 1,  0,  0, 10,  1,  0],
       [ 0,  0,  0,  2,  5,  0],
       [ 1,  0,  0,  0,  1, 10]])

In [17]:
forest=RandomForestClassifier()
forest.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
y_hat2=forest.predict(X_test)

In [19]:
len([1 for i, j in zip(y_hat2, y_test) if i==j])/len(y_test)

0.75

In [20]:
confusion_matrix(y_test, y_hat2)

array([[ 5,  4,  0,  0,  0,  0],
       [ 0, 11,  0,  0,  0,  0],
       [ 2,  1,  3,  0,  0,  3],
       [ 0,  0,  0, 10,  1,  1],
       [ 0,  0,  0,  1,  6,  0],
       [ 0,  0,  0,  0,  2, 10]])